In [1]:
import numpy as np, scipy, pandas as pd
from scipy import stats
rng = np.random.default_rng(seed = 105)
stats.truncnorm.random_state = rng
from symMaps.__init__ import *
from symMaps.lpSys import _adjF
_numtypes = (int,float,np.generic)

# LPSys

The ```LPSys``` is designed for Linear Programming problems.

In [2]:
self = LPSys()

Add some time indices:

In [3]:
t = pd.Index(range(2), name = 't')
i = pd.Index(range(10,13), name = 'i')
j = pd.Index(range(20,24), name = 'j')

Add some variables defined over these indices:

In [4]:
self.v = {'x': t,
          'y': pd.MultiIndex.from_product([t,i]),
          'z': CPI([pd.MultiIndex.from_product([t,i]), j]),
          'k': None} # scalar

## 1. Add 1d coefficients

Coefficients are added as ```GpyDict``` instances. These store dictionaries of ```Gpy``` symbols. We add by specifying (name, parameter) where:

*Example: Specify that cost parameter on part of 'x' is just ones* 

In [5]:
self.lp['c'][('x[t=0]','x')] = pd.Series(1, index = t[0:1]) # Note: The first entry is a unique identifier, the second identifies name of gpy

*Example: Specify a parameter for the scalar variable 'k'.*

In [6]:
self.lp['c']['k'] = 10 # Note: For scalars, the key has to match the symbol name in self.v

In [49]:
self.db['k'] = 10

In [58]:
idxF = pd.Index(['coal','natgas','biomass'], name = 'idxF')
idxEm = pd.Index(['CO2'], name = 'idxEm')
pFuel = pd.Series(np.random.uniform(0,1,len(idxF)), index = idxF)
μEm = pd.Series(np.random.uniform(1,2,len(idxF)), index = pd.MultiIndex.from_product([idxF, idxEm]))
τEm = pd.Series(10, index = idxEm)

In [66]:
(μEm * τEm).groupby('idxF').sum()

idxF
biomass    12.681848
coal       16.458878
natgas     18.817524
dtype: float64

In [59]:
pFuel 

idxF
coal       0.391235
natgas     0.045268
biomass    0.944347
dtype: float64

In [51]:
self.db('k')

10

*Note: We can also add a scalar condition by defining a ```Gpy``` instance with name 'k' and then add with a different key. Or we can add as a tuple:*

In [7]:
self.lp['c']['otherNameForK'] = Gpy.c(10, name = 'k') # in this case, we specify separately that the Gpy symbol should have the name 'k', so the key in the GpyDict can be anything.
self.lp['c'][('otherNameForK','k')] = 10 # equivalent to statement above
self.lp['c'].__delitem__('otherNameForK') # remove again (we have already added it properly above).

*Example*: Add full vector for $x$

In [8]:
self.lp['c']['x'] = pd.Series(np.random.uniform(0, 1, len(self.v['x'])), index = self.v['x'], name = 'x')

*Example*: Add partial vector for $z$. This leaves the rest of the model sparse.

In [9]:
self.lp['c'][('zPart1','z')] = pd.Series(np.random.uniform(1, 2, 10), index = self.v['z'][:10])

**Compilation:**

Create mapping from pandas indices declared in  ```self.v``` to the global index (of stacked variables). This is stored in ```self.maps['v']```:

In [10]:
self.compileMaps() # c
self.maps['v']['y'] # mapping from pandas domains of symbol 'y' to global integer index (indices 2-7 in the global index).

t  i 
0  10    2
   11    3
   12    4
1  10    5
   11    6
   12    7
dtype: int64

We set up vector of coefficients $c$ by calling:

In [11]:
self.compileParams()
self.out['c']

array([ 0.63831963,  0.09743627,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.56342536,  1.82343967,
        1.18324424,  1.03287169,  1.93089709,  1.26348467,  1.99317248,
        1.87441603,  1.77581215,  1.50697846,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , 10.        ])

#### Bounds:

Bounds are defined in exact same way as cost coefficients.

*Lower bound on some variable*

In [12]:
self.lp['l']['x'] = pd.Series(1, index = t, name = 'x')

*Similar for upper bound:*

In [13]:
self.lp['u'][('zPart2','z')] = pd.Series(100, index = self.v['z'][:20])
self.sparse_bounds()

<COOrdinate sparse array of dtype 'int64'
	with 22 stored elements and shape (33, 2)>

Or dense versions:

In [14]:
self.dense_bounds()

array([[  1.,  nan],
       [  1.,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan, 100.],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan]])

#### Constants $b_{eq}$ and $b_{ub}$:

Added in exactly the same way as other 1d coefficients.

## 2. Matrices: Adding coefficients $a_{s,p}$

Ultimately, coefficients in matrices ```A_eq, A_ub``` are added using ```AMatrix``` instances stored in ```AMDict```. We  then move on to a couple of different ways to initialize these ```AMatrix``` coefficients. First, we add some constraints to work over:

In [15]:
self.eq = {'sclr': None,
           'vec_t': t, # entire t index
           'vec_tx0': t[1:], # t index except initial value
           'vec_tj': self.v['z'].droplevel('i').unique()} # combination of t,j that is used in z index 
self.compileMaps()

### 2.1. ```AMDict```

```AMatrix``` instances are simple objects with 6 attributes: 
* name: (unique identifier used to navigate dictionary ```AMDict```),
* values: np.array (or similar) with coefficients to be applied.
* v: name of variable the cofficient applies to, 
* constr: name of constraint the coefficient applies to,
* vIdx: ```pd.Index``` corresponding to variable domains. If None = scalar. 
* constrIdx: ```pd.Index``` corresponding to index comains. If None = scalar constraint.

*Note: ```v, vIdx``` should be consistent with domains registered in ```self.v ``` and similar for (```constr, constrIdx```, in ```self.eq```).*

*Example: Constrained defined over $t$, variable defined over $t,i$:*

In [16]:
constr, v = 'vec_t', 'y' # names 
name = f'{constr}{v}' # unique identifier for this part
series = pd.Series(1, index = self.v[v])
self.initA_infer(name, series = series, v = v, constr = constr, attr = 'eq')

The indices for variable and constraints are stored:

In [17]:
self.lp['A_eq'][name].vIdx

MultiIndex([(0, 10),
            (0, 11),
            (0, 12),
            (1, 10),
            (1, 11),
            (1, 12)],
           names=['t', 'i'])

In [18]:
self.lp['A_eq'][name].constrIdx # note: Indices are not necessarily unique, but rather fits the length of the coefficient that is added 

Index([0, 0, 0, 1, 1, 1], dtype='int64', name='t')

In [19]:
self.lp['A_eq'][name].values # note: Indices are not necessarily unique, but rather fits the length of the coefficient that is added 

array([1, 1, 1, 1, 1, 1])

*Example: Scalar constraint, random coefficients over variable $z$:*

In [20]:
constr, v = 'sclr', 'z'
name = f'{constr}{v}' # unique identifier, doesn't have to be added with syntax 
series = pd.Series(1, index = self.v[v])
self.initA_infer(name, series = series, v = v, constr = constr, attr = 'eq')

In this case, the index represents variable domains, the constraint index is simply None:

In [21]:
self.lp['A_eq'][name].constrIdx is None

True

### 2.2. ```Lags and leads```

Define a parameter over the relevant domains again:

In [22]:
constr, v = 'vec_tx0', 'y'
name = f'{constr}{v}'
series = pd.Series(np.random.uniform(0,1, len(self.v[v])), index = self.v[v])
lag, level = 1,'t'

The following presents a number of equivalent ways of adding this coefficient with lagged index $t-1$:

*VERSION 1 (default):* Add parameter defined full index  $t\in \lbrace 0,1\rbrace$. When the index is lagged
* We do not keep parameters defined for constraint index $t=2$ (```fkeep = False```); this removes forwarded indices beyond original domains).
* We do not keep parameters defined for constraint index $t=0$ (```bfill = False```); when the constraint index $t=0$, the parameter index evaluates to $-1$, which is not specified in the coefficient parameter. When ```bfill = False``` we drop this as well.

In [23]:
self.initA_lag(name, series, v = v, constr = constr, attr = 'eq', lag = lag, level = level, fkeep = False, bfill = False)

The result is that this is defined for $t=1$ for the constraint index, $t=0$ in the variable index (as it evaluates $t-1$), with values from ```series[0, i]```:

In [24]:
self.lp['A_eq'][name].vIdx

MultiIndex([(0, 10),
            (0, 11),
            (0, 12)],
           names=['t', 'i'])

In [25]:
self.lp['A_eq'][name].constrIdx

Index([1, 1, 1], dtype='int64', name='t')

In [26]:
self.lp['A_eq'][name].values

array([0.59853408, 0.81278364, 0.97327969])

*VERSION 2:* Define parameter over relevant index for the variable, that is for $t = 0$ only, but keep forwarded indices (as $t=1$ maps to the correct constraint index):
* We keep parameters defined for constraint index $t=1$ (```fkeep = True```).
* We do not keep parameters defined for constraint index $t=0$ (```bfill = False```); when the constraint index $t=0$, the parameter index evaluates to $-1$, which is not specified in the coefficient parameter. When ```bfill = False``` we drop this as well.

In [27]:
series_v2 = adj.rc_pd(series, t[:-1]) # don't use terminal state T
self.initA_lag(name, series_v2, v = v, constr = constr, attr = 'eq', lag = lag, level = level, fkeep = True, bfill = False)

### 2.3. ```Roll index```

Akin to Lead, but assumes that the index is circular. With index values (1,2,3), rolling this 1 step yields (3,1,2). As this always maps to indices in the index, we do need to take a stance on domains outside the specified ones here: 

In [28]:
constr = 'vec_t' # the constraint should contain all t with circular references.
name = f'{constr}{v}'
self.initA_roll(name, series, v=v, constr = constr, attr = 'eq', roll = -1, level = 't')

## 2.4. Broadcast and add

In the examples outlined above, we added the conditions with coefficients defined over a combination of constraint and variable indices. The following specifies a more "lazy" way of 
adding coefficients that utilizes broadcasting to relevant domains. Consider the somewhat generic linear equation:

$$\begin{align}
    Constr[e]: \qquad b[e] = a[e,x]\times v[x],
\end{align}$$
that defines an equation defined over domains $e$ as a linear combination of a variable $v$ defined over domains $x$. The domains $e,x$ may be 0d (scalars), 1d (pd.Index), or multidimensional (nd). The domains $e,x$ may be the same, overlap partially, or not at all. Similarly, overlapping domains may be related through a mapping e.g. like a roll or a lag of the relevant index.

The following method for adding coefficients $a[e,x]$ is based on the following steps:
1. Specify name of constraint (```constr```) and domains $e$ (```cIdx```). If no domain is provided (```cIdx = None```), we default to using the domain stored in ```self.eq[constr]```.
2. Specify name of variable (```v```) and domains $x$ (```vIdx```). If no domain is provided (```vIdx = None```), we default to using the domain stored in ```self.v[v]```.
    * Note: If we want to apply a mapping to the domains $x$ relative to $e$, we apply this before broadcasting domains. For instance, say that both constraint and variable is defined over $t$. Lagging the $t$ index with 1, the broadcasting matches elements $t$ in the constraint index to $t-1$ in the variable index.
    * From 1-2. we use the ```Broadcast.idx``` method to establish a full combined index. 
3. Now, specify parameter as a pd.Series defined over a subset of combined domains of $e,x$. The parameter is broadcasted to the full index using ```Broadcast.seriesToIdx```.

Main method is implemented as ```self.lazyA```. Version that lags/rolls the variable index is defined as ```self.lazyA_lag```, ```self.lazyA_roll```.

## 3. Extract solution

```__call__``` method extracts part of np.array:

In [29]:
x = np.random.uniform(0,1,self.len['v'])
self(x, 'x')

array([0.74407446, 0.21708886])

```get``` method returns ```pd.Series``` (or scalar):

In [41]:
self.lp['c']

In [35]:
self.v

{'x': RangeIndex(start=0, stop=2, step=1, name='t'),
 'y': MultiIndex([(0, 10),
             (0, 11),
             (0, 12),
             (1, 10),
             (1, 11),
             (1, 12)],
            names=['t', 'i']),
 'z': MultiIndex([(0, 10, 20),
             (0, 10, 21),
             (0, 10, 22),
             (0, 10, 23),
             (0, 11, 20),
             (0, 11, 21),
             (0, 11, 22),
             (0, 11, 23),
             (0, 12, 20),
             (0, 12, 21),
             (0, 12, 22),
             (0, 12, 23),
             (1, 10, 20),
             (1, 10, 21),
             (1, 10, 22),
             (1, 10, 23),
             (1, 11, 20),
             (1, 11, 21),
             (1, 11, 22),
             (1, 11, 23),
             (1, 12, 20),
             (1, 12, 21),
             (1, 12, 22),
             (1, 12, 23)],
            names=['t', 'i', 'j']),
 'k': None}

In [30]:
self.get(x, 'x')

t
0    0.744074
1    0.217089
Name: x, dtype: float64

Remove all but equality constraint, add one inequality constraint, and add bounds on all variables to make the solution feasible and bounded:

In [31]:
delitems = [k for k in (self.lp['A_eq'].symbols) if k != 'vec2vec']
[self.lp['A_eq'].__delitem__(k) for k in delitems];
self.compileMaps()
self.compileParams();
self.out['bounds'] = np.vstack([np.zeros(self.len['v']), np.full(self.len['v'], 10)]).T # this update of the bounds is deleted if we run a standard "compileParams" statement. 
sol = scipy.optimize.linprog(**self.out)

The solution is unloaded as dictionaries of pandas series elements in the following blocks:
* ```self.unloadSolX``` returns dictionary with solution for variables.
* ```self.unloadSolDualEq, self.unloadSolDualUb ``` returns dictionary with shadow values (duals) for the equality and inequality contraints, respectively.
* ```self.unloadSolDualLower, self.unloadSolDualUpper``` returns dictionary with shadow values (duals) for the lower/upper bounds on variables.

The ```self.unloadSol``` method combines all of the above in one:

In [32]:
solDict = self.unloadSol(sol)
solDict.keys() # a look at the symbols added from this stage

dict_keys(['x', 'y', 'z', 'k', 'λeq_sclr', 'λeq_vec_t', 'λeq_vec_tx0', 'λeq_vec_tj', 'λl_x', 'λl_y', 'λl_z', 'λl_k', 'λu_x', 'λu_y', 'λu_z', 'λu_k'])